In [ ]:
from bokeh.io import output_notebook
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.Tools.aliTreePlayer import *
from bokeh.io import curdoc
import numpy as np
import os
import sys
from ROOT import TFile, gSystem
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.MLpipeline.NDFunctionInterface import  *
import plotly.express as px
output_notebook()
from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib.pyplot as plt
from bokeh.transform import factor_cmap, factor_mark

from scipy.stats import norm
import matplotlib.mlab as mlab
import scipy.stats as stats
import math

In [ ]:
def simulatePandaDCA():
    sigma0=0.1
    sigma1=1
    qPtSlope=0.1
    tglSlope=0.1
    entries=1000
    # qPt,tgl.mdEdx.alpha, dCA
    range=([-5,5],[-1,1],[0,1],[0,2*np.pi],[-10*sigma0-1,10*sigma0+1])
    bins=[50,20,20,12,100]
    H, edges = np.histogramdd(sample=np.array([[0, 0, 0, 0, 0]]), bins=bins, range=range)
    indexH = np.arange(H.size)
    indexC = np.unravel_index(indexH, bins)
    qPtCenter = (edges[0][indexC[0]]+edges[0][indexC[0]+1])*.5
    tgl       = (edges[1][indexC[1]]+edges[1][indexC[1]+1])*.5
    mdEdx     = (edges[2][indexC[2]]+edges[2][indexC[2]+1])*.5
    alpha     = (edges[3][indexC[3]]+edges[3][indexC[3]+1])*.5
    #
    valueMean=qPtCenter*qPtSlope+tgl*tglSlope
    value = edges[4][indexC[4]]
    valueSigma = sigma0 * np.sqrt(1 + sigma1 * mdEdx * qPtCenter * qPtCenter)
    weight = np.exp(-(value-valueMean)**2 / (2 * valueSigma * valueSigma))
    weightPoisson = np.random.poisson(weight * entries, H.size)
    H = weightPoisson
    df = pd.DataFrame({"qPtCenter": qPtCenter, "tglCenter": tgl, "mdEdxCenter": mdEdx, "V":value, "alphaCenter":alpha, "mean":valueMean,"rms":valueSigma,"weight": H})
    return df

In [ ]:
%%time
df = simulatePandaDCA()
df["Vcorr"]=df["V"]-df["mean"]
df.head(5)

In [ ]:
output_file("dcaCompressionDemo.html")    
optionsAll={"colorZvar":"mdEdxCenter"}
df["Vcorr"]=df["V"]-df["mean"]
figureArray = [
    [['qPtCenter'], ['mean'], optionsAll],
    [['tglCenter'], ['mean'], optionsAll],
    [['qPtCenter'], ['tglCenter'],{"color": "red", "size": 2, "colorZvar": "mean", "varZ": "mean"}], 
    [['V'], ['weight'], optionsAll],
    [['V'], ['histo'], {"range_min":-2, "range_max":2,"weights":"weight","nbins":100}],
    [['Vcorr'], ['histo'], {"range_min":-2, "range_max":2,"weights":"weight","nbins":100}],
]

widgetParams=[
    ['range', ['tglCenter']],
    ['range', ['qPtCenter']],
    ['range', ['mdEdxCenter']],
    ['range', ['alphaCenter']],
]
tooltips = [("qPtMean", "@qPtMean")]
widgetLayoutDesc=[ [0,1],[2,3], {'sizing_mode':'scale_width'} ]
figureLayoutDesc=[
    [0,1, {'plot_height':150,'x_visible':1}],
    [2,3, {'plot_height':150,'x_visible':1}],
    [4,5, {'plot_height':250,'x_visible':1}],
    {'plot_height':250,'sizing_mode':'scale_width',"legend_visible":True}
]
fig=bokehDrawSA.fromArray(df.sample(100000), "rms>0", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc, nPointRender=10000,rescaleColorMapper=True)

In [ ]:
(8*1000000*4)/10**6

In [ ]:
df.shape

In [ ]:
(24000000* 8*4)/10**6